In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [2]:
def convert_datatype(dataset):
    #Setting dtypes

    dataset["Batting Innings"] = dataset["Batting Innings"].astype(int)
    dataset["Not out"] = dataset["Not out"].astype(int)
    dataset["Runs"] = dataset["Runs"].astype(int)
    dataset["High Score"] = dataset["High Score"].astype(int)
    dataset["Batting Average"] = dataset["Batting Average"].astype(float)
    dataset["Balls Faced"] = dataset["Balls Faced"].astype(int)
    dataset["Batting SR"] = dataset["Batting SR"].astype(float)
    dataset["100's"] = dataset["100's"].astype(int)
    dataset["50's"] = dataset["50's"].astype(int)
    dataset["0's"] = dataset["0's"].astype(int)
    dataset["4s"] = dataset["4s"].astype(int)
    dataset["6s"] = dataset["6s"].astype(int)
    dataset["Bowling Innings"] = dataset["Bowling Innings"].astype(int)
    dataset["Overs"] = dataset["Overs"].astype(float)
    dataset["Maidens"] = dataset["Maidens"].astype(int)
    dataset["Bowling Runs"] = dataset["Bowling Runs"].astype(int)
    dataset["Wickets"] = dataset["Wickets"].astype(int)
    dataset["Bowling Average"] = dataset["Bowling Average"].astype(float)
    dataset["Bowling Economy"] = dataset["Bowling Economy"].astype(float)
    dataset["Bowling SR"] = dataset["Bowling SR"].astype(float)
    dataset["4W Haul"] = dataset["4W Haul"].astype(int)
    dataset["5W Haul"] = dataset["5W Haul"].astype(int)
    dataset["Year"] = dataset["Year"].astype(int)
    
    return dataset

In [3]:
def read_player_data():
    path = r'C:\Users\khand\Desktop\IPL Predictions\data_version4'
    all_files = glob.glob(path + "/*.xlsx")
    columns=['Player', 'Matches', 'Batting Innings', 'Not out', 'Runs', 'High Score',
       'Batting Average', 'Balls Faced', 'Batting SR', "100's", "50's", "0's",
       '4s', '6s', 'Bowling Innings', 'Overs', 'Maidens', 'Bowling Runs',
       'Wickets', 'BBI', 'Bowling Average', 'Bowling Economy', 'Bowling SR',
       '4W Haul', '5W Haul', 'Catches', 'Stumpings']
    
    li = []

    for filename in all_files:
        df = pd.read_excel(filename, index_col=None, header=0, sheet_name="Sheet6")
        df.columns = columns
        df["Year"] = filename[-9: -5]                                      #Extracting Year from filename 
        df["Team"] = filename[-14: -10].strip('4\\')                     #Extracting Team Name from filename 
        df = df.replace('-','0')
        df['High Score'] = df['High Score'].astype('str').str.strip('*')
        df["Team"] = df["Team"].str.upper()
        li.append(df)

    all_years_player_performance = pd.concat(li, axis=0, ignore_index=True)
    all_years_player_performance.drop("BBI", axis=1, inplace=True) #BBI information is not needed
    all_years_player_performance = convert_datatype(all_years_player_performance)
    
    return all_years_player_performance

In [4]:
all_years_player_performance = read_player_data()

In [5]:
all_years_player_performance["Team"].unique()

array(['CSK', 'DC', 'DD', 'GL', 'KKR', 'KTK', 'KXIP', 'MI', 'PWI', 'RCB',
       'RPS', 'RR', 'SRH'], dtype=object)

In [6]:
all_years_player_performance

,Player,Matches,Batting Innings,Not out,Runs,High Score,Batting Average,Balls Faced,Batting SR,100's,...,Wickets,Bowling Average,Bowling Economy,Bowling SR,4W Haul,5W Haul,Catches,Stumpings,Year,Team
0,MEK Hussey,4,3,1,168,116,84.00,100,168.00,1,...,0,0.00,0.00,0.0,0,0,2,0,2008,CSK
1,M Ntini,9,2,1,11,11,11.00,18,61.11,0,...,7,34.57,6.91,30.0,1,0,1,0,2008,CSK
2,ML Hayden,4,4,1,189,81,63.00,131,144.27,0,...,0,0.00,0.00,0.0,0,0,0,0,2008,CSK
3,M Muralitharan,15,1,0,0,0,0.00,3,0.00,0,...,11,36.72,6.96,31.6,0,0,4,0,2008,CSK
4,MS Dhoni,16,14,4,414,65,41.40,310,133.54,0,...,0,0.00,0.00,0.0,0,0,6,0,2008,CSK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,DJ Hooda,11,7,1,64,20,10.66,63,101.58,0,...,1,21.00,10.50,12.0,0,0,10,0,2019,SRH
2019,Basil Thampi,3,1,1,1,1,0.00,1,100.00,0,...,0,0.00,9.16,0.0,0,0,0,0,2019,SRH
2020,S Nadeem,3,0,0,0,0,0.00,0,0.00,0,...,2,45.00,10.00,27.0,0,0,0,0,2019,SRH
2021,RK Bhui,1,1,0,7,7,7.00,12,58.33,0,...,0,0.00,0.00,0.0,0,0,0,0,2019,SRH


In [7]:
def ahp_bat_strength(df):
    """
    Calculates Player Batting Strength using the formula calculated using AHP.
    """
    
    df["AHP_bat"] = 0.388726074*df["Batting Average"] + 0.260099468*df["Batting Innings"]\
                  + 0.175428513*df["Batting SR"] + 0.083438652*df["50's"]\
                  + 0.055018035*df["100's"] + 0.037289258*df["0's"]
    

def ahp_bowl_strength(df):
    """
    Calculates Player Bowling Strength using the formula calculated using AHP.
    """
    
    df["AHP_bowl"] = 0.387508576*df["Overs"] + 0.281308382*df["Bowling Economy"]\
                  + 0.158764794*df["Wickets"] + 0.073609038*df["Bowling Average"]\
                  + 0.067861912*df["Bowling SR"] + 0.030947298*df["4W Haul"]
    
ahp_bat_strength(all_years_player_performance)
ahp_bowl_strength(all_years_player_performance)

In [8]:
ahp_only = all_years_player_performance[["Player", "Year", "Team", "AHP_bat", "AHP_bowl"]]
ahp_only

,Player,Year,Team,AHP_bat,AHP_bowl
0,MEK Hussey,2008,CSK,62.960297,0.000000
1,M Ntini,2008,CSK,15.516622,21.229464
2,ML Hayden,2008,CSK,51.006089,0.000000
3,M Muralitharan,2008,CSK,0.297389,31.027177
4,MS Dhoni,2008,CSK,43.328253,0.000000
...,...,...,...,...,...
2018,DJ Hooda,2019,SRH,23.821834,6.247653
2019,Basil Thampi,2019,SRH,17.802951,7.226888
2020,S Nadeem,2019,SRH,0.000000,11.762869
2021,RK Bhui,2019,SRH,13.213927,0.000000


In [9]:
list(ahp_only.groupby(['Year', 'Team']))

[((2008, 'CSK'),
               Player  Year Team    AHP_bat   AHP_bowl
  0        MEK Hussey  2008  CSK  62.960297   0.000000
  1           M Ntini  2008  CSK  15.516622  21.229464
  2         ML Hayden  2008  CSK  51.006089   0.000000
  3    M Muralitharan  2008  CSK   0.297389  31.027177
  4          MS Dhoni  2008  CSK  43.328253   0.000000
  5           MS Gony  2008  CSK  41.600867  31.374974
  6          SK Raina  2008  CSK  43.803658   7.496077
  7         JA Morkel  2008  CSK  42.001490  26.542504
  8          L Balaji  2008  CSK   0.297389  20.105676
  9       S Badrinath  2008  CSK  41.413532   0.000000
  10         JDP Oram  2008  CSK  34.091645  15.122419
  11         PA Patel  2008  CSK  32.093562   0.000000
  12  Joginder Sharma  2008  CSK  29.443473  16.902540
  13       SP Fleming  2008  CSK  31.938249   0.000000
  14       P Amarnath  2008  CSK   0.000000  16.409333
  15         S Vidyut  2008  CSK  32.580741   6.461916
  16    CK Kapugedera  2008  CSK  15.092305   5.

In [10]:
ahp_only["AHP_bat"].min(), ahp_only["AHP_bat"].max()

(0.0, 87.40383884089)

In [11]:
ahp_only["AHP_bowl"].min(), ahp_only["AHP_bowl"].max()

(0.0, 40.39204295606)

In [12]:
ahp_only.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2023 entries, 0 to 2022
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    2023 non-null   object 
 1   Year      2023 non-null   int32  
 2   Team      2023 non-null   object 
 3   AHP_bat   2023 non-null   float64
 4   AHP_bowl  2023 non-null   float64
dtypes: float64(2), int32(1), object(2)
memory usage: 71.2+ KB


In [13]:
ahp_only.iloc[:, 3:]

,AHP_bat,AHP_bowl
0,62.960297,0.000000
1,15.516622,21.229464
2,51.006089,0.000000
3,0.297389,31.027177
4,43.328253,0.000000
...,...,...
2018,23.821834,6.247653
2019,17.802951,7.226888
2020,0.000000,11.762869
2021,13.213927,0.000000


## We need to scale the values the AHP columns

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
ahp_only.iloc[:, 3:] = scaler.fit_transform(ahp_only.iloc[:, 3:])

C:\Users\khand\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [15]:
ahp_only

,Player,Year,Team,AHP_bat,AHP_bowl
0,MEK Hussey,2008,CSK,0.720338,0.000000
1,M Ntini,2008,CSK,0.177528,0.525585
2,ML Hayden,2008,CSK,0.583568,0.000000
3,M Muralitharan,2008,CSK,0.003402,0.768151
4,MS Dhoni,2008,CSK,0.495725,0.000000
...,...,...,...,...,...
2018,DJ Hooda,2019,SRH,0.272549,0.154675
2019,Basil Thampi,2019,SRH,0.203686,0.178919
2020,S Nadeem,2019,SRH,0.000000,0.291217
2021,RK Bhui,2019,SRH,0.151182,0.000000


In [16]:
ahp_only["AHP_bat"].min(), ahp_only["AHP_bat"].max()

(0.0, 1.0)

In [17]:
ahp_only["AHP_bowl"].min(), ahp_only["AHP_bowl"].max()

(0.0, 1.0)

In [18]:
ahp_only["AHP_bat+bowl"] = ahp_only["AHP_bat"] + ahp_only["AHP_bowl"]
ahp_only

C:\Users\khand\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Player,Year,Team,AHP_bat,AHP_bowl,AHP_bat+bowl
0,MEK Hussey,2008,CSK,0.720338,0.000000,0.720338
1,M Ntini,2008,CSK,0.177528,0.525585,0.703113
2,ML Hayden,2008,CSK,0.583568,0.000000,0.583568
3,M Muralitharan,2008,CSK,0.003402,0.768151,0.771553
4,MS Dhoni,2008,CSK,0.495725,0.000000,0.495725
...,...,...,...,...,...,...
2018,DJ Hooda,2019,SRH,0.272549,0.154675,0.427224
2019,Basil Thampi,2019,SRH,0.203686,0.178919,0.382605
2020,S Nadeem,2019,SRH,0.000000,0.291217,0.291217
2021,RK Bhui,2019,SRH,0.151182,0.000000,0.151182


## Selecting the best 11 players from each team and each year

In [19]:
list(ahp_only.groupby(['Year', 'Team']))

[((2008, 'CSK'),
               Player  Year Team   AHP_bat  AHP_bowl  AHP_bat+bowl
  0        MEK Hussey  2008  CSK  0.720338  0.000000      0.720338
  1           M Ntini  2008  CSK  0.177528  0.525585      0.703113
  2         ML Hayden  2008  CSK  0.583568  0.000000      0.583568
  3    M Muralitharan  2008  CSK  0.003402  0.768151      0.771553
  4          MS Dhoni  2008  CSK  0.495725  0.000000      0.495725
  5           MS Gony  2008  CSK  0.475962  0.776761      1.252723
  6          SK Raina  2008  CSK  0.501164  0.185583      0.686747
  7         JA Morkel  2008  CSK  0.480545  0.657122      1.137667
  8          L Balaji  2008  CSK  0.003402  0.497763      0.501166
  9       S Badrinath  2008  CSK  0.473818  0.000000      0.473818
  10         JDP Oram  2008  CSK  0.390047  0.374391      0.764438
  11         PA Patel  2008  CSK  0.367187  0.000000      0.367187
  12  Joginder Sharma  2008  CSK  0.336867  0.418462      0.755329
  13       SP Fleming  2008  CSK  0.365410  0

In [20]:
ahp_final = ahp_only.groupby(['Year','Team'])['Year', 'Team', 'Player','Team', 'AHP_bat', 'AHP_bowl', 'AHP_bat+bowl'].apply(lambda x: x.nlargest(11, columns=['AHP_bat+bowl']))
ahp_final

C:\Users\khand\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


Year Team           Player Team   AHP_bat  AHP_bowl  \
Year Team                                                             
2008 CSK  5     2008  CSK          MS Gony  CSK  0.475962  0.776761   
          7     2008  CSK        JA Morkel  CSK  0.480545  0.657122   
          3     2008  CSK   M Muralitharan  CSK  0.003402  0.768151   
          10    2008  CSK         JDP Oram  CSK  0.390047  0.374391   
          12    2008  CSK  Joginder Sharma  CSK  0.336867  0.418462   
...              ...  ...              ...  ...       ...       ...   
2019 SRH  2005  2019  SRH      JM Bairstow  SRH  0.595689  0.000000   
          2016  2019  SRH  Shakib Al Hasan  SRH  0.223642  0.310768   
          2012  2019  SRH         KK Ahmed  SRH  0.003402  0.508980   
          2007  2019  SRH        MK Pandey  SRH  0.489348  0.000000   
          2017  2019  SRH           S Kaul  SRH  0.006378  0.463873   

                AHP_bat+bowl  
Year Team                     
2008 CSK  5         1.252723  
          7         1.137667  
          3         0.771553  
          10        0.764438  
          12        0.755329  
...                      ...  
2019 SRH  2005      0.595689  
          2016      0.534411  
          2012      0.512382  
          2007      0.489348  
          2017      0.470251  

[1100 rows x 7 columns]

In [21]:
ahp_final.head(50)

Year  Team           Player  Team   AHP_bat  AHP_bowl  \
Year Team                                                              
2008 CSK  5    2008   CSK          MS Gony   CSK  0.475962  0.776761   
          7    2008   CSK        JA Morkel   CSK  0.480545  0.657122   
          3    2008   CSK   M Muralitharan   CSK  0.003402  0.768151   
          10   2008   CSK         JDP Oram   CSK  0.390047  0.374391   
          12   2008   CSK  Joginder Sharma   CSK  0.336867  0.418462   
          0    2008   CSK       MEK Hussey   CSK  0.720338  0.000000   
          1    2008   CSK          M Ntini   CSK  0.177528  0.525585   
          6    2008   CSK         SK Raina   CSK  0.501164  0.185583   
          2    2008   CSK        ML Hayden   CSK  0.583568  0.000000   
          15   2008   CSK         S Vidyut   CSK  0.372761  0.159980   
          8    2008   CSK         L Balaji   CSK  0.003402  0.497763   
     DC   195  2008    DC         RP Singh    DC  0.282602  0.698248   
          186  2008    DC    Shahid Afridi    DC  0.425628  0.454580   
          185  2008    DC  Y Venugopal Rao    DC  0.467031  0.405681   
          182  2008    DC        A Symonds    DC  0.675328  0.165294   
          184  2008    DC        SB Styris    DC  0.289543  0.533751   
          197  2008    DC        SB Bangar    DC  0.248582  0.479351   
          190  2008    DC    DP Vijaykumar    DC  0.206662  0.466660   
          183  2008    DC        RG Sharma    DC  0.500277  0.171714   
          188  2008    DC          PP Ojha    DC  0.117559  0.532425   
          198  2008    DC        DNT Zoysa    DC  0.300182  0.321722   
          193  2008    DC       WPUJC Vaas    DC  0.278038  0.329205   
     DD   294  2008    DD      MF Maharoof    DD  0.434438  0.506900   
          288  2008    DD       GD McGrath    DD  0.187712  0.711605   
          289  2008    DD         V Sehwag    DD  0.566228  0.319270   
          297  2008    DD        VY Mahesh    DD  0.159730  0.597306   
          295  2008    DD         R Bhatia    DD  0.328237  0.385063   
          299  2008    DD    Mohammad Asif    DD  0.113405  0.510613   
          300  2008    DD     Shoaib Malik    DD  0.294742  0.277426   
          292  2008    DD         A Mishra    DD  0.240571  0.324330   
          298  2008    DD       PJ Sangwan    DD  0.091110  0.438981   
          285  2008    DD        G Gambhir    DD  0.511873  0.000000   
          287  2008    DD         S Dhawan    DD  0.445206  0.000000   
     KKR  585  2008   KKR         I Sharma   KKR  0.267488  0.632033   
          586  2008   KKR         Umar Gul   KKR  0.481698  0.365526   
          578  2008   KKR        DJ Hussey   KKR  0.417620  0.386200   
          576  2008   KKR       SC Ganguly   KKR  0.399050  0.332502   
          587  2008   KKR       AB Agarkar   KKR  0.301502  0.416224   
          573  2008   KKR      BB McCullum   KKR  0.701343  0.000000   
          577  2008   KKR         AB Dinda   KKR  0.118178  0.552224   
          582  2008   KKR        LR Shukla   KKR  0.394881  0.268462   
          588  2008   KKR         M Kartik   KKR  0.276411  0.336550   
          579  2008   KKR  Mohammad Hafeez   KKR  0.219204  0.263519   
          575  2008   KKR          WP Saha   KKR  0.440312  0.000000   
     KXIP 834  2008  KXIP        PP Chawla  KXIP  0.427916  0.640149   
          828  2008  KXIP        IK Pathan  KXIP  0.353509  0.691523   
          827  2008  KXIP    LA Pomersbach  KXIP  1.000000  0.000000   
          838  2008  KXIP        VRV Singh  KXIP  0.282880  0.678100   
          840  2008  KXIP         JR Hopes  KXIP  0.424120  0.476897   
          839  2008  KXIP     Yuvraj Singh  KXIP  0.471062  0.242995   

               AHP_bat+bowl  
Year Team                    
2008 CSK  5        1.252723  
          7        1.137667  
          3        0.771553  
          10       0.764438  
          12       0.755329  
          0        0.720338  
          1        0.703113  


In [22]:
# ahp_final.to_csv("ahp_final.csv", index=False)